In [48]:
import vtk

In [87]:
import numpy as np
from sgt.core.polydata import Polydata, Centerlines
from vtk.util.numpy_support import vtk_to_numpy as v2n
from vtk.util.numpy_support import numpy_to_vtk as n2v

In [95]:
c = Centerlines()
c.load_polydata('../results/healthy/test/centerlines.vtp')

In [66]:
tmp = vtk.vtkIdList()
c.polydata.GetPointCells(0, tmp)

In [77]:
br_id = c.get_pointdata_array(c.PointDataFields.BRANCHID)
bf_id = c.get_pointdata_array(c.PointDataFields.BIFURCATIONID)

# global node id
gid = c.get_pointdata_array(c.PointDataFields.NODEID)
print(gid)
# outlet points are only connected to one cell (skip inlet point)
ids = vtk.vtkIdList()
outlets = []
for p in range(c.polydata.GetNumberOfPoints()):
    c.polydata.GetPointCells(p, ids)
    print("IDS", ids.GetNumberOfIds())
    if ids.GetNumberOfIds() == 1 and gid[p] != 0:
        print(p)
        print(br_id[p])
        assert br_id[p] != -1, 'bifurcation ' + str(bf_id[p]) + ' is connected to an outlet'
        outlets += [br_id[p]]


[    0     1     2 ... 18900 18901 18902]
IDS 1
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 3
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 3
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 3
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 3
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 2
IDS 

In [78]:
len(outlets)

94

In [96]:
cent_id = v2n(c.polydata.GetPointData().GetArray("CenterlineId"))

In [97]:
br_id = c.get_pointdata_array("BranchId")

In [111]:
for i in range(len(cent_id)):
    if cent_id[i][44] == 1 and cent_id[i].sum() == 1:
        print(i)
        

8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573


In [112]:
cent_id[8500]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)

In [113]:
br_id[8500]

73

In [63]:
for p in range(c.polydata.GetNumberOfPoints()):
    c.polydata.GetPointCells(0, tmp)
    if tmp.GetNumberOfIds() == 1:
            assert br_id[p] != -1, 'bifurcation ' + str(bf_id[p]) + ' is connected to an outlet'
            outlets += [br_id[p]]
    return outlets


In [59]:
tmp.GetNumberOfIds()

1

In [43]:
a = np.arange(10) <= 7
b = np.arange(10) >= 0

In [45]:
a

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False])

In [47]:
np.where(np.logical_and(a, b))[0]

array([0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
pointdata = c.get_pointdata()

res = {}
for arr_name in c.get_pointdata_arraynames():
    res[arr_name] = c.get_pointdata_array(arr_name)

In [5]:
points = c.get_points()
points

array([[-1.96995   ,  7.80802   , -0.148507  ],
       [-1.8847456 ,  7.4823313 ,  0.12432455],
       [-1.7995412 ,  7.1566424 ,  0.3971561 ],
       ...,
       [ 2.959553  , -1.0814046 , -5.21313   ],
       [ 2.8962016 , -1.1369073 , -5.20918   ],
       [ 2.83285   , -1.19241   , -5.20523   ]], dtype=float32)

In [6]:
ids_cent = np.unique(res['BranchId']).tolist()
ids_cent.remove(-1)

In [7]:
ids_cent

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145]

In [8]:
from sgt.core.solver import SolverResults

In [9]:
results = SolverResults.from_csv('../results/healthy/0080_0001/LPN_files/0080_0001.jc.sim.0/branch_results.csv')

In [10]:
results.result_df['name'].unique()

array(['branch0_seg0', 'branch1_seg0', 'branch2_seg0', 'branch3_seg0',
       'branch4_seg0', 'branch4_seg1', 'branch4_seg2', 'branch5_seg0',
       'branch6_seg0', 'branch7_seg0', 'branch8_seg0', 'branch9_seg0',
       'branch9_seg1', 'branch9_seg2', 'branch10_seg0', 'branch10_seg1',
       'branch10_seg2', 'branch11_seg0', 'branch12_seg0', 'branch13_seg0',
       'branch13_seg1', 'branch13_seg2', 'branch14_seg0', 'branch15_seg0',
       'branch15_seg1', 'branch15_seg2', 'branch16_seg0', 'branch17_seg0',
       'branch18_seg0', 'branch18_seg1', 'branch18_seg2', 'branch19_seg0',
       'branch20_seg0', 'branch21_seg0', 'branch22_seg0', 'branch23_seg0',
       'branch24_seg0', 'branch24_seg1', 'branch24_seg2', 'branch25_seg0',
       'branch26_seg0', 'branch27_seg0', 'branch28_seg0', 'branch29_seg0',
       'branch30_seg0', 'branch31_seg0', 'branch32_seg0', 'branch33_seg0',
       'branch34_seg0', 'branch34_seg1', 'branch34_seg2', 'branch35_seg0',
       'branch36_seg0', 'branch36_seg1'

In [11]:
import re
def parse_branches(t):
    new = []
    for x in t:
        new.append(int(re.match("branch([0-9]*)_*", x).group(1)))
        
    return list(np.unique(new))

In [12]:
assert parse_branches(results.result_df['name'].unique()) == ids_cent, "No"

In [15]:
results.result_df['time'].unique()

array([0.        , 0.00070659, 0.00141317, 0.00211976, 0.00282635,
       0.00353293, 0.00423952, 0.00494611, 0.00565269, 0.00635928,
       0.00706587, 0.00777245, 0.00847904, 0.00918563, 0.00989221,
       0.0105988 , 0.01130539, 0.01201197, 0.01271856, 0.01342515,
       0.01413173, 0.01483832, 0.0155449 , 0.01625149, 0.01695808,
       0.01766466, 0.01837125, 0.01907784, 0.01978442, 0.02049101,
       0.0211976 , 0.02190418, 0.02261077, 0.02331736, 0.02402394,
       0.02473053, 0.02543712, 0.0261437 , 0.02685029, 0.02755688,
       0.02826346, 0.02897005, 0.02967664, 0.03038322, 0.03108981,
       0.0317964 , 0.03250298, 0.03320957, 0.03391616, 0.03462274,
       0.03532933, 0.03603592, 0.0367425 , 0.03744909, 0.03815568,
       0.03886226, 0.03956885, 0.04027544, 0.04098202, 0.04168861,
       0.0423952 , 0.04310178, 0.04380837, 0.04451495, 0.04522154,
       0.04592813, 0.04663471, 0.0473413 , 0.04804789, 0.04875447,
       0.04946106, 0.05016765, 0.05087423, 0.05158082, 0.05228

In [16]:
for f in ['flow', 'pressure']:
    # initialize output arrays
    array_f = np.zeros((res['Path'].shape[0], len(results.result_df['time'].unique())))
    n_outlet = np.zeros(res['Path'].shape[0])

In [19]:
array_f.shape

(18903, 999)

In [28]:
from sgt.core.lpn import LPN

In [30]:
lpn = LPN.from_file('../results/healthy/0080_0001/LPN_files/0080_0001.jc.in')

In [27]:
res['Path'][res['BranchId'] == 4]

array([0.        , 0.01978313, 0.04073984, 0.0636905 , 0.08848942,
       0.1148509 , 0.1402708 , 0.16558854, 0.18847835, 0.20864805,
       0.22944584, 0.25017242, 0.26755479, 0.28278643, 0.29891898,
       0.31609707, 0.33234348, 0.34505237, 0.35511787, 0.36657997,
       0.38352106, 0.40483175, 0.42714681, 0.44855461, 0.4722932 ,
       0.50093071, 0.53230209, 0.56350434, 0.59567835, 0.62936704,
       0.66473869, 0.70199975, 0.74133146, 0.7786855 , 0.80717814,
       0.82832962, 0.84832172, 0.86995757, 0.88915045, 0.90900038,
       0.93231003, 0.95404223, 0.97474911, 0.99580803, 1.0188897 ,
       1.04028107, 1.05712349, 1.07071789, 1.07971523, 1.08557108,
       1.09063759, 1.09560198, 1.10085813, 1.10579202, 1.11002768,
       1.11518947, 1.12337119, 1.13442379, 1.14590439, 1.15872349,
       1.17404751, 1.18964973, 1.20579813, 1.22152291, 1.23746538,
       1.25159512, 1.26406291, 1.27603025, 1.28557729, 1.2944655 ,
       1.3054386 , 1.31887096, 1.33644952, 1.36064134, 1.38725

In [23]:
for br in results.result_df['name'].unique():
    v = results.vessel_df(br)

In [38]:
for br_node in lpn.tree_bfs_iterator(lpn.get_branch_tree()):
    for vess_node in br_node.vessel_info:
        print(vess_node)

{'boundary_conditions': {'inlet': 'INFLOW'}, 'vessel_id': 0, 'vessel_length': 1.3805440959686677, 'vessel_name': 'branch0_seg0', 'zero_d_element_type': 'BloodVessel', 'zero_d_element_values': {'C': 1.010376999535209e-05, 'L': 0.24993840441465603, 'R_poiseuille': 0.04048590236868942, 'stenosis_coefficient': 0.0}}
{'junction_coefficient': 0.034317749910164924, 'vessel_id': 1, 'vessel_length': 1.6221170932907283, 'vessel_name': 'branch1_seg0', 'zero_d_element_type': 'BloodVessel', 'zero_d_element_values': {'C': 4.670470898335906e-06, 'L': 0.7464824631496186, 'R_poiseuille': 0.30735843672521673, 'stenosis_coefficient': 0.03431803616271457}}
{'junction_coefficient': 0.03052735647435768, 'vessel_id': 14, 'vessel_length': 0.4700096249069316, 'vessel_name': 'branch10_seg0', 'zero_d_element_type': 'BloodVessel', 'zero_d_element_values': {'C': 1.5047830399804309e-06, 'L': 0.194516073192941, 'R_poiseuille': 0.07202650740965344, 'stenosis_coefficient': 0.030532728483783435}}
{'vessel_id': 15, 'ves

In [39]:
import numpy as np

In [40]:
np.load('../results/healthy/0080_0001/LPN_files/0080_0001.jc.sim.0/branch_results.npy', allow_pickle=True).item()

{'time': array([], dtype=float64),
 'distance': {0: array([0.       , 1.3805441]),
  1: array([0.        , 1.62211709]),
  10: array([0.        , 0.47000962, 1.70627391, 1.73336294]),
  2: array([0.        , 1.84911841]),
  3: array([0.        , 0.96520618]),
  5: array([0.        , 0.03377137]),
  23: array([0.        , 1.97476516]),
  25: array([0.        , 0.53458641]),
  48: array([0.        , 0.27393033]),
  56: array([0.        , 3.97794158]),
  63: array([0.        , 0.32848971, 1.84067212, 2.32858795]),
  67: array([0.        , 0.71398187]),
  88: array([0.        , 1.39330879]),
  102: array([0.        , 0.16948464]),
  134: array([0.        , 1.89646008]),
  11: array([0.        , 0.20013249]),
  16: array([0.        , 0.47049331]),
  28: array([0.        , 0.76709238]),
  30: array([0.        , 1.51055306]),
  33: array([0.        , 0.12287946]),
  45: array([0.        , 1.58579685]),
  52: array([0.        , 0.54669394]),
  57: array([0.        , 1.45020608]),
  60: array([